_Imports_

In [6]:
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
from functions import clean_text
from datasets import load_dataset

tqdm.pandas()

_Load Dataset_

In [7]:
ds = load_dataset("LocalDoc/news_azerbaijan_2")

# huggingface datasetini dataframe-yə çevirdik
df = ds["train"].to_pandas()

# bütün hərfləri kiçik etdik
df = df.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)

# lazımsız sütunları, duplikatları, nulları sildik
df.drop(columns=["id", "date"], inplace=True)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

_Clean Dataset from Unusual Symbols_

In [8]:
# textləri təmizlədik - sadəcə hərflər, rəqəmlər və bir neçə əsas durğu işarəsini saxladıq
for col in ['title', 'text']:
    df[col] = df[col].progress_apply(clean_text)

100%|██████████| 752458/752458 [01:29<00:00, 8395.25it/s] 


_Drop Rows with Outlier Word Counts_

In [9]:
# Çox uzun və çox qısa olan textləri sildik
# XLM modeli maksimum 512 token qəbul edir. Bundan uzun olan textləri özü truncate edir.
# buna görə də çox uzun textlər saxlamağa ehtiyac yoxdur. Çünki model onsuz da onları kəsəcək və bu da contextin yarımçıq qalmasına səbəb olacaq.
# içərisində 1-2 söz olan textləri də saxlamağa ehtiyac yoxdur, çünki onlar kontent cəhətdən qıtdırlar.

df['title_word_count'] = df['title'].str.count(r'\S+')
df['text_word_count'] = df['text'].str.count(r'\S+')

print("BEFORE")
print("Title - Min:", df['title_word_count'].min())
print("Title - Max:", df['title_word_count'].max())
print("Title - Mean:", df['title_word_count'].mean())
print("Title - Median:", df['title_word_count'].median())
print()
print("Text - Min:", df['text_word_count'].min())
print("Text - Max:", df['text_word_count'].max())
print("Text - Mean:", df['text_word_count'].mean())
print("Text - Median:", df['text_word_count'].median())
print("ROW COUNT BEFORE:", len(df))

df = df[
    (df['title_word_count'].between(3, 15)) &
    (df['text_word_count'].between(20, 500))
].reset_index(drop=True)

df['title_word_count'] = df['title'].str.count(r'\S+')
df['text_word_count'] = df['text'].str.count(r'\S+')

print("\nAFTER")
print("Title - Min:", df['title_word_count'].min())
print("Title - Max:", df['title_word_count'].max())
print("Title - Mean:", df['title_word_count'].mean())
print("Title - Median:", df['title_word_count'].median())
print()
print("Text - Min:", df['text_word_count'].min())
print("Text - Max:", df['text_word_count'].max())
print("Text - Mean:", df['text_word_count'].mean())
print("Text - Median:", df['text_word_count'].median())
print("ROW COUNT AFTER:", len(df))

BEFORE
Title - Min: 1
Title - Max: 1309
Title - Mean: 7.328385105879664
Title - Median: 7.0

Text - Min: 2
Text - Max: 73078
Text - Mean: 233.79241366295528
Text - Median: 118.0
ROW COUNT BEFORE: 752458

AFTER
Title - Min: 3
Title - Max: 15
Title - Mean: 7.2987246800049705
Title - Median: 7.0

Text - Min: 20
Text - Max: 500
Text - Mean: 141.78787280974277
Text - Median: 102.0
ROW COUNT AFTER: 643760


_Group and Drop Classes_

In [10]:
# göründüyü kimi datada həddindən artıq çox kateqoriya var və bir xeyli imbalance şəkildə
for category, count in df['category'].value_counts().to_dict().items():
    print(f"{category}:  {count}")

son xəbər:  141529
dünya:  136222
ölkə:  100151
gündəm:  40672
kriminal:  34599
siyasət:  33323
i̇dman:  32472
hadisə:  31705
i̇qtisadiyyat:  19361
şou-biznes:  10405
region:  9472
maraqlı:  8983
yazarlar:  7319
media:  7188
mədəniyyət:  4742
təhsil:  4128
sosial:  3605
yaşam:  3479
səhiyyə:  3044
reportaj:  1843
astrologiya:  1802
texnologiya:  1780
qarabağ xəbərləri:  1582
qalmaqal:  843
müsahibə:  538
ki̇vdf layihələri:  525
mənəviyyat:  464
güney hadisələri:  369
xəbər xətti:  301
video:  251
formula #1:  234
proje:  146
foto fakt:  125
analitika:  100
i̇slamiada:  98
yaxın tarix:  88
rəsmi:  59
mətbuat bugün:  47
oxucu poçtu:  39
hərbi xəbərlər:  33
dünyanın bu üzü:  28
seçki 2024:  24
vaxt axarı:  18
fotosessiya:  9
sözsüz:  8
bizim qonaq:  5
yavru vatanın səsi:  2


In [11]:
# QEYD: aşağıda etdiyim bütün atmalar və ya birləşdirmələr öz intuisiyama görə olmayıb.
# modelin bir dəfə train olunub nəticələrinə baxandan və datanı daha dərin analiz edəndən sonra bu qərara gəldim.
# sadəcə olaraq qarışıqlığın qarşısını almaq üçün ayrıca notebook-da göstərməmişəm

# aşağıda atılan sütunların atılma səbəbləri:
# son xəbər, ölkə, gündəm - bunlar ayrıca bir kateqoriya deyil. Müxtəlif kateqoriyalardan olan xəbərləri özlərində saxlayırlar.
# məsələn. əgər bir xəbər siyasət və ya iqtisadiyyat olsun fərq etmir, yeni çıxıbsa, o həm də son xəbər kateqoriyasında olacaq.
# və bu zaman da model onu digər siniflərlə çox qarışdıracaq
# sübut kimi xlm-roberta modelinin aşağıdakı dəyişikliləri etməmişdən əvvəl train olunmuş halının classification reportunu /results folderinə əlavə etmişəm (result1.png)
# ki̇vdf layihələri - hansısa bir zaman dilimində olub və hazırda bu kimi xəbərlər yoxdur deyə atılıb

drop_cats = ['son xəbər', 'ölkə', 'gündəm', 'ki̇vdf layihələri', 'dünya']
df = df[~df['category'].isin(drop_cats)]

# bu siniflər demək olar ki bir-biri ilə eyni tip xəbərlərə malikdir
# bu səbəbə birləşdirilmişdir
merge_map = {
    'yaşam': ['yaşam', 'maraqlı', 'səhiyyə'],
    'siyasət': ['qarabağ xəbərləri', 'siyasət', 'seçki 2024'],
    'i̇dman': ['i̇dman', 'formula #1', 'i̇slamiada'],
    'region': ['region', 'güney hadisələri'],
    'şou-biznes': ['şou-biznes', 'qalmaqal'],
    'müsahibə': ['müsahibə', 'reportaj'],
    'hadisə': ['hadisə', 'kriminal']
}

for new_cat, old_cats in merge_map.items():
    df['category'] = df['category'].replace(old_cats, new_cat)

# sayı 500-dən az olan dataları class sayını azaltmaq üçün other adı altında yeni bir classda birləşdirdim
category_counts = df['category'].value_counts()
rare_cats = category_counts[category_counts < 500].index
df['category'] = df['category'].apply(lambda x: 'other' if x in rare_cats else x)
df.reset_index(drop=True, inplace=True)

_Undersampling_

In [13]:
# datanın sayı çox olduğundan həm train gec olur, həm də komputerimi yandıra bilər :).
# datanın sayını azaltmaqla bağlı tapşırıqda limit də olmadığı üçün, undersampling edərək biraz imbalanclığı aradan qaldırmağa çalışdım 
df = (df.groupby('category').apply(lambda x: x.sample(n=6000, random_state=42) if len(x) > 6000 else x).reset_index(drop=True))

_Combined Colum for XLM-ROBERTA Model_

In [ ]:
df['input'] = df['title'] + ' - ' + df['text']
df.drop(columns=["title_word_count", "text_word_count"], inplace=True)

_Save Preprocessed Data_

In [ ]:
df.to_parquet("./data/data.parquet", index=False)